In [1]:
import QDMPy.io as Qio
import QDMPy.fit as Qfit
import QDMPy.plot as Qplot
import QDMPy.field as Qfield

import matplotlib
import matplotlib.pyplot as plt
import warnings

In [2]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
%matplotlib widget

### Load raw data

In [3]:
options_dict = {
    # Argus: C:\\src\\QDMPY_test_image
    # Sam: /home/samsc/ResearchData/test_images/
    "base_dir": "/home/samsc/ResearchData/test_images/",
    "filepath": "ODMR - Pulsed_42",
    "custom_output_dir_suffix": "_bin_{total_bin}_{ROI}",
    
    "additional_bins": 8,

    "system_name": "Zyla",

    "ROI": "Full", 
    "ROI_start": [10, 30],
    "ROI_end": [20,40],
    
    "AOI_1_start": [60, 145],
    "AOI_1_end": [70, 155], 
    "AOI_2_start": [20, 20],
    "AOI_2_end": [24, 24], 
    "AOI_3_start": [30, 20],
    "AOI_3_end": [32, 24], 
    "single_pixel_check": [90, 150],
    
    "fit_backend": "scipyfit",
    "fit_backend_comparison": ["scipyfit"],
    "fit_pixels": True, 
    "force_fit": False, 
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipyfit_sub_threads": 2, 

    "fit_functions":  {"linear": 1, "lorentzian": 8},

#     "pos_guess": [2610, 2708, 2805, 2879, 2953, 3020, 3092, 3145], # ODMR - Pulsed with IR_83
    "pos_guess": [2631, 2723, 2787, 2866, 2955, 3022, 3066, 3130], # ODMR - Pulsed_42
    "pos_range": 25,

    "amp_guess": -0.02,
    "amp_bounds": [-0.0300, -0.0003],
    
    "save_fig_type": "pdf",
    
    "freqs_to_use": [1, 1, 1, 1, 1, 1, 1, 1],
    "diamond_ori": "<100>_<110>", # this is one of the funny HPHTs
    
    "bias_mag": 100,
    "bias_theta": 67,
    "bias_phi": -120,
    
#     "field_method": "invert_unvs",
#     "hamiltonian": "bxyz",
    "calc_field_pixels": True,
    "force_field_calc": False
}

In [4]:
options = Qio.load_options(options_dict=options_dict, check_for_prev_result=True)
Qplot.set_mpl_rcparams(options)

### Load data

In [5]:
image, prelim_sweep_list = Qio.load_image_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list = Qio.reshape_dataset(
        options, image, prelim_sweep_list
    )
ROI_PL_fig = Qplot.plot_ROI_PL_image(options, PL_image)

AOI_PL_fig = Qplot.plot_AOI_PL_images(options, PL_image_ROI)

AOI_spectra_fig = Qplot.plot_AOI_spectra(options, sig, ref, sweep_list)

Qio.save_PL_data(options, PL_image, PL_image_ROI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Load reference experiment - note, you can fit in this same notebook!

In [6]:
ref_options_dir = None # if you want to use a reference experiment {ensure you run this even if 'None' as it sets up output dirs etc.}
# below: alternative is to just have the dictionary already loaded (above this is called `pixel_fit_params`)
ref_fit_params, ref_sigmas = Qio.load_reference_experiment_fit_results(options, ref_options=None, ref_options_dir=ref_options_dir)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/io/fit.py:162: UserWarning: No reference experiment options dict provided, continuing without reference.
  warnings.warn(


### Fit AOI, ROI averages, single pixel

In [7]:
fit_model = Qfit.define_fit_model(options)
backend_ROI_results_lst = Qfit.fit_ROI_avg(options, sig_norm, sweep_list, fit_model)
ROI_fit_fig = Qplot.plot_ROI_avg_fits(options, backend_ROI_results_lst)

fit_result_collection_lst = Qfit.fit_AOIs(options, sig_norm, single_pixel_pl, sweep_list, fit_model, backend_ROI_results_lst)
AOI_fit_fig = Qplot.plot_AOI_spectra_fit(options, sig, ref, sweep_list, fit_result_collection_lst, fit_model)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit/_scipyfit.py:35: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # auto detects jupyter


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fit pixels

In [8]:
wanted_roi_result = next(
    filter(
        lambda result: result.fit_backend == options["fit_backend"], backend_ROI_results_lst
    )
) # ROI fit result for chosen fit backend
pixel_fit_params, sigmas = Qfit.get_pixel_fit_results(options, sig_norm, sweep_list, fit_model, wanted_roi_result)

Qio.save_pixel_fit_results(options, pixel_fit_params)
Qio.save_pixel_fit_sigmas(options, sigmas)

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/fit/interface.py:281: UserWarning: Using previous fit results.
  warnings.warn("Using previous fit results.")


### Plot (fit) parameters

In [9]:
c_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "c")
m_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "m")
pos_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "pos")
amp_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "amp")
fwhm_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "fwhm")
residual_fig = Qplot.plot_param_images(options, fit_model, pixel_fit_params, "residual")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
pos_sigma_plot = Qplot.plot_param_images(options, fit_model, sigmas, "pos", errorplot=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Initial guess and fit vs data (flattened)

In [11]:
flat_pos_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "pos", sigmas=sigmas, plot_bounds=False, plot_sigmas=False, errorevery=1000)
flat_resid_fig = Qplot.plot_params_flattened(options, fit_model, pixel_fit_params, wanted_roi_result, "residual", sigmas=sigmas)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Field retrieval

In [12]:
field_res = Qfield.odmr_field_retrieval(options, pixel_fit_params, ref_fit_params)
(sig_bnvs, ref_bnvs, bnvs), (sig_dshifts, ref_dshifts), (sig_params, ref_params, field_params), (sig_field_sigmas, ref_field_sigmas, field_sigmas) = field_res

/home/samsc/src/nv/QDMPy_proj/QDMPy_git/QDMPy/field/interface.py:117: UserWarning: Using previous field calculation.
  warnings.warn("Using previous field calculation.")


In [13]:
bnv_sig_plot = Qplot.plot_bnvs_and_dshifts(options, "sig", sig_bnvs, sig_dshifts)
bnv_ref_plot = Qplot.plot_bnvs_and_dshifts(options, "ref", ref_bnvs, ref_dshifts)
bnvs_plot = Qplot.plot_bnvs_and_dshifts(options, "sig_sub_ref", bnvs, [])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
bfield_sig_plot = Qplot.plot_bfield(options, "sig", sig_params)
bfield_ref_plot = Qplot.plot_bfield(options, "ref", ref_params)
bfield_plot = Qplot.plot_bfield(options, "sig_sub_ref", field_params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
D_sig_plot = Qplot.plot_dshift_fit(options, "sig", sig_params)
D_ref_plot = Qplot.plot_dshift_fit(options, "ref", ref_params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
Bx_fit_flat_plot = Qplot.field_param_flattened(options, "sig", "Bx", sig_params, sigmas=sig_field_sigmas, y_label="B (G)", errorevery=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
field_params, field_sigmas = Qfield.sub_bground_Bxyz(options, field_params, field_sigmas, method="poly", order=1)
bfield_sub_bgound_plot = Qplot.plot_bfield(options, "sig_sub_ref", field_params)
bfield_bground_plot = Qplot.plot_bfield(options, "bground", field_params)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Save and (optionally) view options dict

In [18]:
Qio.save_options(options)
# print(Qio.dict_to_json_str(options))